In [1]:
import pandas as pd
import os
OLD_PATH = "/data/gentrification/buffalo_oarsystem_data"
DATA_DIR = "/data/gentrification/LATEST"
# open data data frame 
open_data_df = pd.read_csv('https://data.buffalony.gov/api/views/kckn-jafw/rows.csv?accessType=DOWNLOAD',low_memory = False)

In [2]:
# we import all function used 
%run all_functions.ipynb

/home/janvolta/.conda/envs/volt_env/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 10005. The TBB threading layer is disabled.
  warnings.warn(problem)


# 1. find all sbls in erie county data

In [3]:
import string
ALL_SBL_FILE = os.path.join(DATA_DIR,"owner_history_htmls")
lists_of_sbls_htmls = os.listdir(ALL_SBL_FILE)
possibilities =  [char for char in string.digits + string.ascii_uppercase] # possible combinations 

if os.path.exists(ALL_SBL_FILE):
    all_sbls = [x.strip().split('.')[0].replace('#','.').replace('+','/') for x in lists_of_sbls_htmls]
    print("ALREADY RAN THIS STEP!")

else:
    all_sbls = []
    for letter in possibilities:
        all_sbls += grab_source(letter,path = DATA_DIR) # creates a new directory and appends htmls files onto the created directories 
   

ALREADY RAN THIS STEP!


# 2. download and compile all owner pages

In [4]:
csv_file_name =  'owner_history.csv'
csv_owner_history_file_path  = os.path.join(DATA_DIR,csv_file_name)
if not os.path.exists(csv_owner_history_file_path): # if it has not downloaded it yet 
    list_of_failed_sbls = html_csv_converter(ALL_SBL_FILE,csv_file_name) # this is to find out SBLS that gave a server error 500
else:
    print('Owner page history has been made')

Owner page history has been made


In [5]:
owner_history_df = pd.read_csv(csv_owner_history_file_path,names= ['Owner','Owner Name','Book-Page/Date' 
                                                  ,'Book-Page/Date identification','SBL'])
owner_history_error_reformat(owner_history_df,open_data_df,inplace = True) # we then remove any noise and corrupted data (i.g. NAN) and replace them with the information open data buffalo has provided 

,Owner,Owner Name,Book-Page/Date,Book-Page/Date identification,SBL,Book-Page
0,Owner,SCIOLI ERALDO/GINA,Book-Page/Date,7096-597 * 3/19/1965 *,123.79-1-1,7096/597
1,Owner,THEN ANDREW J,Book-Page/Date,11138-0710 * 12/4/2007 *,123.79-1-1,11138/0710
2,Owner,PIKUZINSKI MARTYNA &1,Book-Page/Date,6551-381 * 5/27/1960 *,101.49-2-62,6551/381
3,Owner,PIKUZINSKI MARTYNA,Book-Page/Date,9418-83 * 1/24/1985 *,101.49-2-62,9418/83
4,Owner,CARIDINE JOHN L,Book-Page/Date,9955-642 * 12/8/1988 *,101.49-2-62,9955/642
...,...,...,...,...,...,...
308556,Owner,SURDI MARK J,Book-Page/Date,9950-150 * 11/23/1988 *,77.49-1-18,9950/150
308557,Owner,SURDI JOSEPH D,Book-Page/Date,10884-5136 * 3/31/1995 *,77.49-1-18,10884/5136
308558,Owner,SURDI JOSEPH D & BEVERLY,Book-Page/Date,10884-8668 * 4/10/1995 *,77.49-1-18,10884/8668
308559,Owner,BARNES DAVID,Book-Page/Date,11216-6145 * 1/31/2012 *,77.49-1-18,11216/6145


# 3. We scrape the considerations page using the Book-Page column from the owner history 
## link can be found on http://ecclerk.erie.gov/or_wb1/

In [6]:
ALL_CONSIDERATIONS_FILE = os.path.join(DATA_DIR,"considerations_htmls")
if not os.path.exists(ALL_CONSIDERATIONS_FILE): 
    scrape_considerations_page(owner_history_df,DATA_DIR) # creates a 
                                            # new directory called considerations_htmls and puts the newly considerations
else:
    print("CONSIDERATIONS HAVE BEEN SCRAPED")


CONSIDERATIONS HAVE BEEN SCRAPED


# 4. Concat these into a single csv 

In [7]:
file_name = 'complete_search_parcel.csv'
path_joined_concatenated = os.path.join(DATA_DIR,'complete_search_parcel.csv')
if not os.path.exists(path_joined_concatenated):

    # we create a new csv called "complete_search_parcel.csv"
    list_of_failed_deeds = combine_considerations_owner_history(DATA_DIR,file_name)
print("CONSIDERATIONS AND OWNER HISTORY CSVS HAVE BEEN CONCATENATED")
complete_dataframe = pd.read_csv(path_joined_concatenated)
complete_dataframe 

CONSIDERATIONS AND OWNER HISTORY CSVS HAVE BEEN CONCATENATED


,owner name,sbl,consideration,doc type,deed page(search parcel),deed page(public records),date(public records),date(search parcel)
0,SCIOLI ERALDO/GINA,123.79-1-1,$0.00,IMO,7096/597,7096/597,1/1/1900 : : 0,3/19/1965
1,THEN ANDREW J,123.79-1-1,"$45,000.00",D1A,11138/0710,11138/710,12/4/2007 15:27:53,12/4/2007
2,PIKUZINSKI MARTYNA &1,101.49-2-62,$0.00,IMO,6551/381,6551/381,1/1/1900 : : 0,5/27/1960
3,PIKUZINSKI MARTYNA,101.49-2-62,$0.00,IMO,9418/83,9418/83,1/1/1900 : : 0,1/24/1985
4,CARIDINE JOHN L,101.49-2-62,$0.00,4,9955/642,9955/642,3/15/1988 : : 0,12/8/1988
...,...,...,...,...,...,...,...,...
342009,SURDI MARK J,77.49-1-18,$0.00,D99,9950/150,9950/150,11/23/1988 : : 0,11/23/1988
342010,SURDI JOSEPH D,77.49-1-18,"$75,000.00",D1B,10884/5136,10884/5136,3/31/1995 11:25:38,3/31/1995
342011,SURDI JOSEPH D & BEVERLY,77.49-1-18,$1.00,D1B,10884/8668,10884/8668,4/10/1995 11:13:05,4/10/1995
342012,BARNES DAVID,77.49-1-18,"$25,000.00",DEED,11216/6145,11216/6145,1/31/2012 12:46:47,1/31/2012


# 5 Link to location data w/ open data buffalo 



In [8]:
locations_link = link_locations(complete_dataframe,open_data_df)
locations_link

,owner name,sbl,consideration,doc type,deed page(search parcel),deed page(public records),date(public records),date(search parcel),Neighborhood,Location
0,SCIOLI ERALDO/GINA,123.79-1-1,$0.00,IMO,7096/597,7096/597,1/1/1900 : : 0,3/19/1965,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)"
1,THEN ANDREW J,123.79-1-1,"$45,000.00",D1A,11138/0710,11138/710,12/4/2007 15:27:53,12/4/2007,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)"
2,PIKUZINSKI MARTYNA &1,101.49-2-62,$0.00,IMO,6551/381,6551/381,1/1/1900 : : 0,5/27/1960,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
3,PIKUZINSKI MARTYNA,101.49-2-62,$0.00,IMO,9418/83,9418/83,1/1/1900 : : 0,1/24/1985,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
4,CARIDINE JOHN L,101.49-2-62,$0.00,4,9955/642,9955/642,3/15/1988 : : 0,12/8/1988,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
...,...,...,...,...,...,...,...,...,...,...
342009,SURDI MARK J,77.49-1-18,$0.00,D99,9950/150,9950/150,11/23/1988 : : 0,11/23/1988,Riverside,"(42.95494703448873, -78.89975255336158)"
342010,SURDI JOSEPH D,77.49-1-18,"$75,000.00",D1B,10884/5136,10884/5136,3/31/1995 11:25:38,3/31/1995,Riverside,"(42.95494703448873, -78.89975255336158)"
342011,SURDI JOSEPH D & BEVERLY,77.49-1-18,$1.00,D1B,10884/8668,10884/8668,4/10/1995 11:13:05,4/10/1995,Riverside,"(42.95494703448873, -78.89975255336158)"
342012,BARNES DAVID,77.49-1-18,"$25,000.00",DEED,11216/6145,11216/6145,1/31/2012 12:46:47,1/31/2012,Riverside,"(42.95494703448873, -78.89975255336158)"


# 6. Fix Locations and data info as best as we can 

In [9]:
print("Number of Locations we are missing prior: " + str(locations_link['Location'].isna().sum()))
oarsystem_data_path = os.path.join(OLD_PATH,'buffalo_oarsystem_CSVs/combined_tax_data.csv')
oarsystem_df = pd.read_csv(os.path.join(oarsystem_data_path)).rename(columns = {'SBL':'sbl'})
oarsystem_df # note this was scraped before hand; the aim is to take ADDRESS here 

# I took the address from OARSYSTEM, because the data from the real property tax parcel does not have information such as the ZipCode of a SBL 

Number of Locations we are missing prior: 906


,Sale Date,Price,Useable,Arm's Length,Prior Owner,Total Assessed Value,Total Land Value,Deed Book,Deed Page,CENSUS TRACT,BLOCK GROUP,sbl,ADDRESS,NEIGHBORHOOD_open data,LOCATION_open_data
0,5/27/2020,"$177,777.78",NaN,NaN,"Hynes, James J","$160,000.00","$18,000.00",NaN,NaN,7,1,133.66-3-15,"76 SHENANDOAH, Buffalo, 14220",South Park,"(42.839278777194686, -78.80483461698972)"
1,12/1/2006 1:05:21 PM,"$95,000",Yes,Yes,"Hassey Jill A.,","$80,800","$7,200",11122.0,4539.0,7,1,133.66-3-15,"76 SHENANDOAH, Buffalo, 14220",South Park,"(42.839278777194686, -78.80483461698972)"
2,8/23/2001,"$78,000",Yes,Yes,"Kraft Vincent G & W,","$80,800","$7,200",10987.0,6390.0,7,1,133.66-3-15,"76 SHENANDOAH, Buffalo, 14220",South Park,"(42.839278777194686, -78.80483461698972)"
3,5/27/2020,"$54,444.44",NaN,NaN,"McCarter, Kanitra","$49,000.00","$5,000.00",NaN,NaN,170,1,90.55-4-4,"528 CAMBRIDGE, Buffalo, 14215",Delavan Grider,"(42.92906047886588, -78.82341218091284)"
4,10/12/2006 1:36:14 PM,$1,No,No,"Wood Wardell K,","$42,000","$4,300",11120.0,4715.0,170,1,90.55-4-4,"528 CAMBRIDGE, Buffalo, 14215",Delavan Grider,"(42.92906047886588, -78.82341218091284)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300891,5/27/2020,"$101,111.11",NaN,NaN,"Begum, Taj N","$91,000.00","$3,800.00",NaN,NaN,16,4,111.44-4-40,"671 FILLMORE, Buffalo, 14212",Broadway Fillmore,"(42.89160089831186, -78.83883653573429)"
300892,1/3/2013 9:19:40 AM,"$15,000",No,No,"Jackson, Michael","$28,900","$2,800",11236.0,6955.0,16,4,111.44-4-40,"671 FILLMORE, Buffalo, 14212",Broadway Fillmore,"(42.89160089831186, -78.83883653573429)"
300893,1/23/2004 8:37:01 AM,"$12,000",No,No,"Kolniak Henry,","$28,900","$2,800",11067.0,1687.0,16,4,111.44-4-40,"671 FILLMORE, Buffalo, 14212",Broadway Fillmore,"(42.89160089831186, -78.83883653573429)"
300894,5/27/2020,"$360,000.00",NaN,NaN,"Hanley John T & W,","$324,000.00","$89,800.00",NaN,NaN,63.01,3,88.84-3-8,"150 DORCHESTER, Buffalo, 14213",Elmwood Bidwell,"(42.92282606249178, -78.88391630771578)"


In [10]:
fixed_dataframe =  batch_geoencode_missing_location(locations_link,OLD_PATH)
print('-------------------------------------------------------------------------')
print("Number of missing locations now.. : " + str(fixed_dataframe['Location'].isna().sum()))
fixed_dataframe

batch files have been added!
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  136k  100  101k  100 36296   6831   2397  0:00:15  0:00:15 --:--:-- 25960
-------------------------------------------------------------------------
Number of missing locations now.. : 281


,owner name,sbl,consideration,doc type,deed page(search parcel),deed page(public records),date(public records),date(search parcel),Neighborhood,Location
0,SCIOLI ERALDO/GINA,123.79-1-1,$0.00,IMO,7096/597,7096/597,1/1/1900 : : 0,3/19/1965,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)"
1,THEN ANDREW J,123.79-1-1,"$45,000.00",D1A,11138/0710,11138/710,12/4/2007 15:27:53,12/4/2007,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)"
2,PIKUZINSKI MARTYNA &1,101.49-2-62,$0.00,IMO,6551/381,6551/381,1/1/1900 : : 0,5/27/1960,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
3,PIKUZINSKI MARTYNA,101.49-2-62,$0.00,IMO,9418/83,9418/83,1/1/1900 : : 0,1/24/1985,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
4,CARIDINE JOHN L,101.49-2-62,$0.00,4,9955/642,9955/642,3/15/1988 : : 0,12/8/1988,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)"
...,...,...,...,...,...,...,...,...,...,...
342009,SURDI MARK J,77.49-1-18,$0.00,D99,9950/150,9950/150,11/23/1988 : : 0,11/23/1988,Riverside,"(42.95494703448873, -78.89975255336158)"
342010,SURDI JOSEPH D,77.49-1-18,"$75,000.00",D1B,10884/5136,10884/5136,3/31/1995 11:25:38,3/31/1995,Riverside,"(42.95494703448873, -78.89975255336158)"
342011,SURDI JOSEPH D & BEVERLY,77.49-1-18,$1.00,D1B,10884/8668,10884/8668,4/10/1995 11:13:05,4/10/1995,Riverside,"(42.95494703448873, -78.89975255336158)"
342012,BARNES DAVID,77.49-1-18,"$25,000.00",DEED,11216/6145,11216/6145,1/31/2012 12:46:47,1/31/2012,Riverside,"(42.95494703448873, -78.89975255336158)"


In [11]:
new_dataframe = add_multiple_doc_types_column(fix_dates(fixed_dataframe))
new_dataframe 

,owner name,sbl,consideration,doc type,deed page(search parcel),deed page(public records),date(public records),date(search parcel),Neighborhood,Location,Actual Date,more than one doc type
0,SCIOLI ERALDO/GINA,123.79-1-1,$0.00,IMO,7096/597,7096/597,1/1/1900 : : 0,3/19/1965,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)",1965-03-19,False
1,THEN ANDREW J,123.79-1-1,"$45,000.00",D1A,11138/0710,11138/710,12/4/2007 15:27:53,12/4/2007,Hopkins-Tifft,"(42.8565808931454, -78.8250857756282)",2007-12-04,False
2,PIKUZINSKI MARTYNA &1,101.49-2-62,$0.00,IMO,6551/381,6551/381,1/1/1900 : : 0,5/27/1960,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)",1960-05-27,False
3,PIKUZINSKI MARTYNA,101.49-2-62,$0.00,IMO,9418/83,9418/83,1/1/1900 : : 0,1/24/1985,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)",1985-01-24,False
4,CARIDINE JOHN L,101.49-2-62,$0.00,4,9955/642,9955/642,3/15/1988 : : 0,12/8/1988,Genesee-Moselle,"(42.91182584111316, -78.81336114920062)",1988-12-08,True
...,...,...,...,...,...,...,...,...,...,...,...,...
342009,SURDI MARK J,77.49-1-18,$0.00,D99,9950/150,9950/150,11/23/1988 : : 0,11/23/1988,Riverside,"(42.95494703448873, -78.89975255336158)",1988-11-23,True
342010,SURDI JOSEPH D,77.49-1-18,"$75,000.00",D1B,10884/5136,10884/5136,3/31/1995 11:25:38,3/31/1995,Riverside,"(42.95494703448873, -78.89975255336158)",1995-03-31,False
342011,SURDI JOSEPH D & BEVERLY,77.49-1-18,$1.00,D1B,10884/8668,10884/8668,4/10/1995 11:13:05,4/10/1995,Riverside,"(42.95494703448873, -78.89975255336158)",1995-04-10,False
342012,BARNES DAVID,77.49-1-18,"$25,000.00",DEED,11216/6145,11216/6145,1/31/2012 12:46:47,1/31/2012,Riverside,"(42.95494703448873, -78.89975255336158)",2012-01-31,False


# 7. filter out home/apartments 

In [12]:

if os.path.exists('filtered_data.csv'):
    print('filtered data already exists')
    filtered_datav = pd.read_csv('filtered_data.csv')
else:
    filtered_datav, failed_files, anomolies = filter_apartments(new_dataframe,DATA_DIR)
    filtered_datav.to_csv('filtered_data.csv',index = False)

filtered data already exists


# 8. Construct Features 

In [ ]:
%run all_functions.ipynb
feature_construction_for_random_forest = random_classifier_data_pre_processing(filtered_datav,open_data_df)


# tasks layout
3. download all tax history pages
4. concat these into a single csv
5. link to location data w/ open buffalo
6. fix locations and date info as best we can
7. filter out non home/apartments
8. construct features